# 可視化

`./scatter_preprocess.ipynb`で作成した：

- `all_res.csv`
- `umamusume.csv`

を利用して，Plotlyで散布図を描く．

## 環境構築

In [1]:
# Notebook初期設定
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [3]:
# plotly関連
!pip install plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

     |████████████████████████████████| 23.9 MB 8.6 MB/s            


In [4]:
DIR_IN = '../data/scatter_preprocess'
DIR_OUT = '../data/scatter_plot'

In [5]:
FN_ALL = 'all_res.csv'
FN_UM = 'umamusume.csv'

In [6]:
DISTANCE_CLASSES = [
    'short',
    'mile',
    'intermediate',
    'long'
]

In [39]:
INDICES= {
    'turf_index': '芝適性',
    'dart_index': 'ダート適性',
    'short_index': '短距離適性',
    'mile_index': 'マイル適性',
    'intermediate_index': '中距離適性',
    'long_index': '長距離適性',
    'nige_index': '逃げ適性',
    'senko_index': '先行適性',
    'sashi_index': '差し適性',
    'oikomi_index': '追い込み適性',
}

In [8]:
SUBPLOT_TITLES = [
    '短距離（1,600m未満）',
    'マイル（1,600m～2,000m未満）',
    '中距離（2,000m～2,500m未満）',
    '長距離（2,500m以上）'
]

In [55]:
X_TITLE = 'レース全体の平均の速さ[km/h]'
Y_TITLE = '上り3ハロンの平均の速さ[km/h]'

## 可視化

In [63]:
def make_hover_text(
        horse_name, horse_age, 
        race_name, turf, dart, date, distance,
        seconds_total, seconds_3f,
        speed_total, speed_3f,
        arrival_order, prize):
    
    if turf:
        race_type = '芝'
    elif dart:
        race_type = 'ダート'
    else:
        race_type = ''
        
    text = f'''<b>{horse_name}</b> ({horse_age}歳) <br><br>
    レース：{race_name} ({date}, {race_type}, {distance}m)<br>
    タイム： {seconds_total} 秒 (上り: {seconds_3f} 秒) <br>
    平均の速さ: {speed_total:.4} km/h （上り: {speed_3f:.4} km/h） <br>
    着順：{arrival_order}<br>
    賞金：{prize}万円
    '''
    return text

In [64]:
# ホバー用のカラムを追加
def add_hover_text_to_df(df):
    df_new = df.copy()
    df_new['hover_text'] = \
        df_new[
            ['horse_name', 'horse_age', 
             'race_name', 'turf', 'dart', 'date', 'distance', 
             'seconds_total', 'seconds_3f',
             'speed_total', 'speed_3f',
             'arrival_order', 'prize']].apply(
        lambda x: make_hover_text(*x), axis=1)
    return df_new

In [65]:
def get_min_and_max_of_col(df, col):
    """dfのcolのminとmaxを取得"""
    col_min = df[col].min() * 0.9
    col_max = df[col].max() * 1.1
    return (col_min, col_max)

In [66]:
def make_df_for_plot(df, dc, color_col, asc):
    """scatter描画用のdfを生成"""
    df_tmp = \
        df[df['distance_class']==dc].reset_index(drop=True)
    df_tmp = df_tmp.sort_values(
        color_col, ascending=asc, ignore_index=True)
    return df_tmp

In [67]:
def add_scatter_trace_to_fig(
        fig, x, y, color, text, name, i,
        opacity=1., symbol='circle', size=10, 
        hover=True, linecolor='White'):
    """figに対しscatterを追加"""
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode='markers',
            marker_symbol=symbol,
            marker_size=size,
            opacity=opacity,
            hoverinfo='text' if hover else 'skip',
            marker={
                'color': color,
                'coloraxis':'coloraxis',
                'line':{
                    'color': linecolor,
                    'width': 1},
            },
            text=text,
            hovertemplate='%{text}' if hover else None,
            name=name,
        ),
    i//2+1, i%2+1)

In [68]:
def update_colorbar_of_fig(fig, color_title):
    """figのカラーバーに関する設定"""
    fig.update_layout(
        showlegend=False,
        coloraxis_colorbar={'title': color_title},
        )

In [69]:
def update_axis_ranges_of_fig(fig, x_min, x_max, y_min, y_max):
    """figの描画範囲を更新"""
    fig.update_xaxes(range=[x_min, x_max])
    fig.update_yaxes(range=[y_min, y_max])

In [70]:
def update_axis_titles_of_fig(
        fig, x_title=X_TITLE, y_title=Y_TITLE):
    """figの各軸名を修正"""
    fig.update_xaxes(title_text=x_title)
    fig.update_yaxes(title_text=y_title)

In [71]:
def subplots_scatter_by_distance_class(
        df, color_col='prize', color_title='獲得賞金', asc=True):
    """距離区分ごとにsubplotでscatterを描画"""
    fig = make_subplots(
        rows=2, cols=2, subplot_titles=SUBPLOT_TITLES)
    x_min, x_max = get_min_and_max_of_col(df, 'speed_total')
    y_min, y_max = get_min_and_max_of_col(df, 'speed_3f')
    for i, dc in enumerate(DISTANCE_CLASSES):
        df_tmp = make_df_for_plot(df, dc, color_col, asc)
        add_scatter_trace_to_fig(
            fig, x=df_tmp['speed_total'], y=df_tmp['speed_3f'],
            color=df_tmp[color_col], text=df_tmp['hover_text'],
            name=dc, i=i)
    update_colorbar_of_fig(fig, color_title)
    update_axis_ranges_of_fig(
        fig, x_min=x_min, x_max=x_max, 
        y_min=y_min, y_max=y_max)
    update_axis_titles_of_fig(fig)
    return fig

In [72]:
def subplots_two_scatters_by_distance_class(
        df, df_star, color_col='prize', color_title='獲得賞金', asc=True):
    """距離区分ごとにsubplotでscatterを描画
    ただしdf_starの結果は☆でプロット"""
    fig = make_subplots(
        rows=2, cols=2, subplot_titles=SUBPLOT_TITLES)
    x_min, x_max = get_min_and_max_of_col(df, 'speed_total')
    y_min, y_max = get_min_and_max_of_col(df, 'speed_3f')
    for i, dc in enumerate(DISTANCE_CLASSES):
        df_tmp = make_df_for_plot(df, dc, color_col, asc)
        df_star_tmp = make_df_for_plot(df_star, dc, color_col, asc)
        # 背景表示用
        add_scatter_trace_to_fig(
            fig, x=df_tmp['speed_total'], y=df_tmp['speed_3f'],
            color=df_tmp[color_col], text=df_tmp['hover_text'],
            name=dc, i=i, opacity=0.3, hover=False)
        # 注目したいデータ用
        add_scatter_trace_to_fig(
            fig, x=df_star_tmp['speed_total'], y=df_star_tmp['speed_3f'],
            color=df_star_tmp[color_col], text=df_star_tmp['hover_text'],
            name=dc, i=i, symbol='star', size=25)
    update_colorbar_of_fig(fig, color_title)
    update_axis_ranges_of_fig(
        fig, x_min=x_min, x_max=x_max, 
        y_min=y_min, y_max=y_max)
    update_axis_titles_of_fig(fig)
    return fig

In [73]:
def subplots_three_scatters_by_distance_class(
        df, df_star, df_tri, color_col='prize', 
        color_title='獲得賞金', asc=True):
    """距離区分ごとにsubplotでscatterを描画
    ただしdf_starの結果は☆で，df_triの結果は△でプロット"""
    fig = make_subplots(
        rows=2, cols=2, subplot_titles=SUBPLOT_TITLES)
    x_min, x_max = get_min_and_max_of_col(df, 'speed_total')
    y_min, y_max = get_min_and_max_of_col(df, 'speed_3f')
    for i, dc in enumerate(DISTANCE_CLASSES):
        df_tmp = make_df_for_plot(df, dc, color_col, asc)
        df_star_tmp = make_df_for_plot(df_star, dc, color_col, asc)
        df_tri_tmp = make_df_for_plot(df_tri, dc, color_col, asc)
        # 背景表示用
        add_scatter_trace_to_fig(
            fig, x=df_tmp['speed_total'], y=df_tmp['speed_3f'],
            color=df_tmp[color_col], text=df_tmp['hover_text'],
            name=dc, i=i, opacity=0.3, hover=False)
        # 注目したいデータ（☆）
        add_scatter_trace_to_fig(
            fig, x=df_star_tmp['speed_total'], y=df_star_tmp['speed_3f'],
            color=df_star_tmp[color_col], text=df_star_tmp['hover_text'],
            name=dc, i=i, symbol='star', size=25, opacity=0.8)
        # 注目したいデータ（□）
        add_scatter_trace_to_fig(
            fig, x=df_tri_tmp['speed_total'], y=df_tri_tmp['speed_3f'],
            color=df_tri_tmp[color_col], text=df_tri_tmp['hover_text'],
            name=dc, i=i, symbol='triangle-up', size=25, opacity=0.8)
    update_colorbar_of_fig(fig, color_title)
    update_axis_ranges_of_fig(
        fig, x_min=x_min, x_max=x_max, 
        y_min=y_min, y_max=y_max)
    update_axis_titles_of_fig(fig)
    return fig

### 全レース結果の散布図（`scatter_all.html`）

In [74]:
df_all = pd.read_csv(os.path.join(DIR_IN, FN_ALL))
# ホバー表示用のカラムを追加
df_all = add_hover_text_to_df(df_all)

In [75]:
df_all.head().T

,0,1,2,3,4
race_id,198605020811,198605020811,198605020811,198605020811,198605020811
date,1986-05-11,1986-05-11,1986-05-11,1986-05-11,1986-05-11
place,東京,東京,東京,東京,東京
race_name,第36回安田記念(G1),第36回安田記念(G1),第36回安田記念(G1),第36回安田記念(G1),第36回安田記念(G1)
distance,1600,1600,1600,1600,1600
dart,False,False,False,False,False
dart_cond,None,None,None,None,None
turf,True,True,True,True,True
turf_cond,良,良,良,良,良
steeple,False,False,False,False,False


In [76]:
fig = subplots_scatter_by_distance_class(df_all)
fig.write_html(f'{DIR_OUT}/scatter_all.html')

### ウマ娘ごと（`horses/scatter_{horse_name}.html`）

In [77]:
df_all = pd.read_csv(os.path.join(DIR_IN, FN_ALL))
# ホバー表示用のカラムを追加
df_all = add_hover_text_to_df(df_all)

In [78]:
df_um = pd.read_csv(os.path.join(DIR_IN, FN_UM))

In [79]:
# 以下いずれかを満たすウマ娘をプロット対象とする
## 条件1. ウマ娘に登場していること
ums_um = set(df_um['horse_name'].unique())

## 条件2. 獲得賞金トップ50に入っていること
df_tmp = \
    df_all.groupby('horse_id')[['horse_name', 'prize']].\
    agg({'horse_name':max, 'prize':sum}).\
    sort_values(by='prize', ascending=False).reset_index().head(50)
ums_prize = set(df_tmp['horse_name'].unique())

## 和集合
ums = ums_um | ums_prize

In [80]:
# 上記のうち，馬名が重複する馬が存在しないか検証
df_tmp = df_all[df_all['horse_name'].isin(ums)].groupby('horse_name')\
    ['horse_id'].nunique().reset_index()
assert (df_tmp['horse_id'] > 1).sum() == 0

In [81]:
# 上記のうち，df_all中に存在しない馬
ums - set(df_all['horse_name'].unique())

{'シンボリルドルフ', 'ハルウララ', 'マルゼンスキー', 'ミスターシービー'}

- シンボリルドルフ：1985-12-22の有馬記念を最後に表舞台から姿を消す
- ハルウララ：高知競馬場を中心に活躍したため，中央競馬における出走記録なし
- マルゼンスキー：1977-7-24の短距離Sを最後に表舞台から姿を消す
- ミスターシービー：1985-4-29の天皇賞（春）を最後に表舞台から姿を消す

In [82]:
ums = ums & set(df_all['horse_name'].unique())
# プロット対象とするウマ娘+競走馬の数
len(ums)

98

In [83]:
for um in tqdm(ums):
    # 注目するレース結果のみ抽出
    df_star = \
        df_all[df_all['horse_name']==um].\
        reset_index(drop=True)
    fig = subplots_two_scatters_by_distance_class(df_all, df_star)
    fig.write_html(f'{DIR_OUT}/horses/scatter_{um}.html')

  0%|          | 0/98 [00:00<?, ?it/s]

### タイトルごと（`titles/scatter_{title}.html`）

In [84]:
df_all = pd.read_csv(os.path.join(DIR_IN, FN_ALL))
# ホバー表示用のカラムを追加
df_all = add_hover_text_to_df(df_all)

In [85]:
df_tmp = df_all.groupby('title')['grade'].unique().reset_index()
df_tmp['grade'] = df_tmp['grade'].apply(lambda x: '_'.join(x))
df_tmp['fn'] = df_tmp[['title', 'grade']].apply(
    lambda x: f'{x[1]}_{x[0]}', axis=1)
title2fn = df_tmp.groupby('title')['fn'].first().to_dict()

In [86]:
for title, fn in tqdm(title2fn.items()):
    # 注目するレース結果のみ抽出
    ## 賞金を獲得した馬のみ
    df_star = \
        df_all[(df_all['title']==title)&(df_all['prize']>0)].\
        reset_index(drop=True)
    fig = subplots_two_scatters_by_distance_class(df_all, df_star)
    fig.write_html(f'{DIR_OUT}/titles/scatter_{fn}.html')

  0%|          | 0/274 [00:00<?, ?it/s]

### 適性ごと（`indices/scatter_{index}_A.html`）

In [87]:
df_all = pd.read_csv(os.path.join(DIR_IN, FN_ALL))
# ホバー表示用のカラムを追加
df_all = add_hover_text_to_df(df_all)

In [88]:
df_um = pd.read_csv(os.path.join(DIR_IN, FN_UM))

In [89]:
for index, index_fn in tqdm(INDICES.items()):
    ums = df_um[df_um[index]=='A']['horse_name'].unique()
    df_star = \
        df_all[df_all['horse_name'].isin(ums)].\
        reset_index(drop=True)
    fig = subplots_two_scatters_by_distance_class(
            df_all, df_star)
    fn = f'{index_fn}_A'
    fig.write_html(
        f'{DIR_OUT}/indices/scatter_{fn}.html')

  0%|          | 0/10 [00:00<?, ?it/s]

### 比較（`vs/scatter_{horse_name_0}_{horse_name_1}.html`）

In [90]:
ums_vs = [
    ['トウカイテイオー', 'メジロマックイーン'],
    ['スペシャルウィーク', 'サイレンススズカ'],
    ['ウオッカ', 'ダイワスカーレット'],
    ['キタサンブラック', 'サトノダイヤモンド'],
    ['ライスシャワー', 'メジロマックイーン'],
    ['トウカイテイオー', 'ツインターボ'],
]

In [91]:
df_all = pd.read_csv(os.path.join(DIR_IN, FN_ALL))
# ホバー表示用のカラムを追加
df_all = add_hover_text_to_df(df_all)

In [92]:
for [um0, um1] in tqdm(ums_vs):
    # 注目するレース結果のみ抽出
    df_star = \
        df_all[df_all['horse_name']==um0].\
        reset_index(drop=True)
    df_tri = \
        df_all[df_all['horse_name']==um1].\
        reset_index(drop=True)
    fig = subplots_three_scatters_by_distance_class(
            df_all, df_star, df_tri)
    fig.write_html(f'{DIR_OUT}/vs/scatter_{um0}_{um1}.html')

  0%|          | 0/6 [00:00<?, ?it/s]